<a href="https://colab.research.google.com/github/nathanr8/Netflix_Recommendation_System/blob/main/Rating_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [43]:
import tensorflow as tf### models
import numpy as np### math computations
import sklearn### machine learning library
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import pandas as pd
import datetime
import pathlib
import io
import os
import time
import random
from google.colab import files
from PIL import Image
import albumentations as A
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import matplotlib.cm as cm
from google.colab import drive

In [3]:
train_directory = "/content/dataset/Netflix Dataset/Netflix Dataset/train"
validation_directory = "/content/dataset/Netflix Dataset/Netflix Dataset/test"

In [5]:
! rm -r ~/.kaggle
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 /root/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d shivamb/netflix-shows

  0% 0.00/1.34M [00:00<?, ?B/s]
100% 1.34M/1.34M [00:00<00:00, 117MB/s]


In [7]:
!unzip "/content/netflix-shows.zip" -d "/content/dataset/"

Archive:  /content/netflix-shows.zip
  inflating: /content/dataset/netflix_titles.csv  


In [44]:
netflix_overall=pd.read_csv("/content/dataset/netflix_titles.csv")
netflix_overall

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [45]:
netflix_overall.count()

show_id         8807
type            8807
title           8807
director        6173
cast            7982
country         7976
date_added      8797
release_year    8807
rating          8803
duration        8804
listed_in       8807
description     8807
dtype: int64

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [47]:
tfidf_Var = TfidfVectorizer(stop_words='english')

In [48]:
netflix_overall['description'] = netflix_overall['description'].fillna('')

In [49]:
tfidf_matrix = tfidf_Var.fit_transform(netflix_overall['description'])

In [51]:
tfidf_matrix.shape

(8807, 18895)

In [52]:
from sklearn.metrics.pairwise import linear_kernel


In [53]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [54]:
indices = pd.Series(netflix_overall.index, index=netflix_overall['title']).drop_duplicates()

In [56]:
def recommender_system(title, cosine_sim=cosine_sim):
  index = indices[title]
  similarity_scores = list(enumerate(cosine_sim[index]))
  similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
  similarity_scores = similarity_scores[1:6]
  movie_indices = [i[0] for i in similarity_scores]
  return netflix_overall['title'].iloc[movie_indices]

In [57]:
recommender_system('To Be of Service')

1307              The Resurrection of Gavin Stone
7294                           Let There Be Light
7740                           Pick of the Litter
5761    LEGO Jurassic World: The Indominus Escape
2403                                       Menahi
Name: title, dtype: object

In [58]:
filled = netflix_overall.fillna('')

In [59]:
def clean_data(x):
        return str.lower(x.replace(" ", ""))

In [60]:
features=['title','director','cast','listed_in','description']
filled = filled[features]

In [61]:
for feature in features:
    filled[feature] = filled[feature].apply(clean_data)

filled.head(20)

,title,director,cast,listed_in,description
0,dickjohnsonisdead,kirstenjohnson,,documentaries,"asherfathernearstheendofhislife,filmmakerkirst..."
1,blood&water,,"amaqamata,khosingema,gailmabalane,thabangmolab...","internationaltvshows,tvdramas,tvmysteries","aftercrossingpathsataparty,acapetownteensetsou..."
2,ganglands,julienleclercq,"samibouajila,tracygotoas,samueljouy,nabihaakka...","crimetvshows,internationaltvshows,tvaction&adv...","toprotecthisfamilyfromapowerfuldruglord,skille..."
3,jailbirdsneworleans,,,"docuseries,realitytv","feuds,flirtationsandtoilettalkgodownamongthein..."
4,kotafactory,,"mayurmore,jitendrakumar,ranjanraj,alamkhan,ahs...","internationaltvshows,romantictvshows,tvcomedies",inacityofcoachingcentersknowntotrainindia’sfin...
5,midnightmass,mikeflanagan,"katesiegel,zachgilford,hamishlinklater,henryth...","tvdramas,tvhorror,tvmysteries",thearrivalofacharismaticyoungpriestbringsglori...
6,mylittlepony:anewgeneration,"robertcullen,joséluisucha","vanessahudgens,kimikoglenn,jamesmarsden,sofiac...",children&familymovies,equestria'sdivided.butabright-eyedherobelieves...
7,sankofa,hailegerima,"kofighanaba,oyafunmikeogunlano,alexandraduah,n...","dramas,independentmovies,internationalmovies","onaphotoshootinghana,anamericanmodelslipsbacki..."
8,thegreatbritishbakingshow,andydevonshire,"melgiedroyc,sueperkins,maryberry,paulhollywood","britishtvshows,realitytv",atalentedbatchofamateurbakersfaceoffina10-week...
9,thestarling,theodoremelfi,"melissamccarthy,chriso'dowd,kevinkline,timothy...","comedies,dramas",awomanadjustingtolifeafteralosscontendswithafe...
